TEST ML_composer 


In [18]:
import os
from GS_composer import *

In [19]:
import os
print(os.getcwd())
#os.chdir("F:/project/sugarcane_disease/")
#os.chdir("O:/project/sugarcane_disease/")
os.getcwd()
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
user_profile = "C:/Users/uqcche32/OneDrive - The University of Queensland/"
#os.chdir("E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/")
os.chdir("C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/")

C:\Users\uqcche32\OneDrive - The University of Queensland\PhD\HPC_Results\Sugarcane_disease\ML


In [20]:
from ClassModel import *
from CustomLayers import *

In [21]:
from Functions import *

In [107]:
class p_args:
    
    def __init__(self):
        
        self.ped = user_profile +"/PhD/data/sugarcane_disease/subset/disease_subset"
        self.pheno = user_profile +"PhD/data/sugarcane_disease/subset/disease_subset.phen"
        self.mpheno = 1
        self.index = user_profile +"PhD/data/sugarcane_disease/subset/disease_subset.index"
        self.model = "MultiHead Attention LNN"
        self.annotation =  user_profile +"PhD/data/sugarcane_disease/subset/disease_subset.anno"
        self.trait = "smut"
        self.output = "./Attention_Test_win"
        
        self.load = None
        self.save = True
        self.plot = True
        
        self.round = 1
        self.epoch = 10
        self.rank =False
        self.lr = 0.0001
        self.batch = 32
        self.mean = False
        self.width = 64
        self.depth = 2
        self.quiet = 1
        self.num_heads = 1
        self.residual = True
        self.embedding = 12
        self.data_type = "numerous"
        self.activation = "linear"
        self.classes = 0
        
        
args = p_args()

In [108]:
config = None


In [109]:
if args.output[0] == "/":
    locat = '/' + args.output.strip('/') + '/'
else:
    locat = args.output.strip('/') + '/'
if not os.path.exists(locat):
    os.mkdir(locat)

In [110]:
composer = ML_composer()
composer._model
composer.silence_mode = 2

In [111]:
#composer.get_data(config,args)
composer.args = args

In [112]:
composer.get_data(config,args=args)

1    939
3    940
4    940
2    940
5    943
Name: 2, dtype: int64
Get genotype shape: (4702, 100)
   7   8   9   10  11  12  13  14  15
1   0   2   0   1   2   1   0   1   2
2   1   2   1   1   2   1   0   1   1
3   1   2   0   0   1   0   1   1   2
4   1   2   0   0   1   0   1   1   2
5   0   2   1   1   2   0   2   1   1
6   0   2   0   1   2   1   0   1   2
7   1   2   1   1   2   1   2   1   2
8   0   2   0   1   1   0   1   0   2
9   1   2   1   1   2   1   2   1   2
Running data check
GENO
FAM
PHENO
INDEX
SNPs in annotation file are not ordered by map file
Got LD shape:
(100, 6)


In [113]:
#composer.prepare_model()
#composer._model["INIT_MODEL"].init_model()

In [114]:
args.model

'MultiHead Attention LNN'

In [115]:
index_ref = composer.prepare_cross_validate()

In [116]:
print("Cross-validate: {}".format(index_ref[0]))

Cross-validate: ([2, 3, 4, 5], [1])


In [117]:
train_idx,valid_idx = index_ref[0]

In [118]:
composer.prepare_training(train_idx,valid_idx)
#composer.batchSize = 64

Overall population: 4702
759 individuals need to be removed due to the miss phenotype
Filtered population: 3943
Mean of train phenotype: 4.715129635707253
Use raw phenotype as the target
16    1.0
39    3.0
44    3.0
47    2.0
92    3.0
Name: 2, dtype: float64


In [119]:
#composer._raw_data["GENO"].iloc[:,6:].describe()

In [120]:
composer.compose(train_idx,valid_idx)

USE LocalRealFormer MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (3047, 100, 1)
The transformed SNP shape: (3047,) <dtype: 'float32'>
USE LocalRealFormer MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (896, 100, 1)
The transformed SNP shape: (896,) <dtype: 'float32'>
Train status:
Epochs:  10
Repeat(Round):  1
Got input shape: (100, 1)
(None, 100, 1)
[TensorShape([None, 6, 12])]
[TensorShape([None, 6, 12]), TensorShape([None, 6, 12])]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
snp_block_layer (SNPBlockLayer) (None, 6, 12)        1200        input_l

F:\miniconda\envs\tf25\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


96/96 [==============================] - 5s 54ms/step - loss: 7.3979 - acc: 0.0309 - val_loss: 4.2612 - val_acc: 0.0190
Epoch 2/10
96/96 [==============================] - 5s 49ms/step - loss: 6.2461 - acc: 0.0309 - val_loss: 3.9405 - val_acc: 0.0190
Epoch 3/10
96/96 [==============================] - 5s 50ms/step - loss: 5.8031 - acc: 0.0309 - val_loss: 3.9215 - val_acc: 0.0190
Epoch 4/10
96/96 [==============================] - 5s 49ms/step - loss: 5.6474 - acc: 0.0309 - val_loss: 4.1741 - val_acc: 0.0190
Epoch 5/10
96/96 [==============================] - 5s 51ms/step - loss: 5.5341 - acc: 0.0309 - val_loss: 3.9421 - val_acc: 0.0190
Epoch 6/10
96/96 [==============================] - 5s 51ms/step - loss: 5.4813 - acc: 0.0309 - val_loss: 4.0100 - val_acc: 0.0190
Epoch 7/10
96/96 [==============================] - 5s 50ms/step - loss: 5.3918 - acc: 0.0309 - val_loss: 4.1119 - val_acc: 0.0190
Epoch 8/10
96/96 [==============================] - 5s 51ms/step - loss: 5.3454 - acc: 0.0309 

F:\miniconda\envs\tf25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Model saved.
Resource check:
Total memory: 31.7264 GB
Currently using memory: 1.7217 GB
Ratio of used memory: 30.8000  %
Number of CPU node:  16
Plotting the training process...
C:\Users\uqcche32\OneDrive - The University of Queensland\PhD\HPC_Results\Sugarcane_disease\ML\Attention_Test_win/smut_MultiHeadAttentionLNN_smut
Plotting loss history...
Plot name:  C:\Users\uqcche32\OneDrive - The University of Queensland\PhD\HPC_Results\Sugarcane_disease\ML\Attention_Test_win/smut_MultiHeadAttentionLNN_smut/smut_MultiHeadAttentionLNN_1.png
Result:
  Trait      TrainSet ValidSet                  Model  Test_Accuracy  \
0  smut  [2, 3, 4, 5]      [1]  MultiHeadAttentionLNN        0.31512   

   Valid_Accuracy       MSE   Runtime  
0        0.109187  3.905827  0.833333  


In [121]:
composer._model["TRAINED_MODEL"].save("./test_model")

#import tensorflow as tf 

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

INFO:tensorflow:Assets written to: ./test_model\assets


F:\miniconda\envs\tf25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Default GPU Device:/device:GPU:0


In [ ]:
1.5//1

In [ ]:
os.getcwd()

In [ ]:
index_ref = composer.prepare_cross_validate()
i = 1
for train_idx,valid_idx in index_ref:
    print("Cross-validate: {}".format(i))
    composer.prepare_training(train_idx,valid_idx)
    composer.compose(train_idx,valid_idx)
    i+=1